In [12]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

# Imports

In [13]:
import ray

from ray.tune.registry import register_env
from ray.tune.logger import pretty_print

# from ray.rllib.algorithms.apex_ddpg import ApexDDPGConfig
from ray.rllib.algorithms.dqn import DQNConfig, DQNTFPolicy, DQNTorchPolicy

# Env

In [14]:
def env_creator(render_mode="rgb_array", cycles=200):
    from src.world import world_utils
    env = world_utils.env(render_mode=render_mode, max_cycles=cycles)

    return env

# Parameters

In [15]:
stop_iters = 20
stop_timesteps = 100000
stop_reward = 50.0

# Ray config

In [16]:
ray.init()
env = env_creator()
register_env("tagworld", env_creator)

2023-03-10 16:59:29,790	INFO worker.py:1553 -- Started a local Ray instance.


In [18]:
obs_space = env.observation_space
act_space = env.action_space

In [22]:
policies = {
    "dqn_policy": (
    DQNTorchPolicy,
    obs_space,
    act_space,
    {},
    ),
}

In [19]:
def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    return "dqn_policy"
        # if agent_id % 2 == 0:
            # return "ppo_policy"
        # else:

In [23]:
dqn_config = (
        DQNConfig()
        .environment("tagworld")
        .framework("torch")
        # disable filters, otherwise we would need to synchronize those
        # as well to the DQN agent
        .rollouts(observation_filter="MeanStdFilter")
        .training(
            model={"vf_share_layers": True},
            n_step=3,
            gamma=0.95,
        )
        .multi_agent(
            policies=policies,
            policy_mapping_fn=policy_mapping_fn,
            policies_to_train=["dqn_policy"],
        )
        # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
        .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
    )

dqn = dqn_config.build()

2023-03-10 17:01:28,217	INFO algorithm.py:506 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


ValueError: Traceback (most recent call last):
  File "/run/media/nishant/Data/Work/uni/thesis/repo/env/lib/python3.9/site-packages/ray/rllib/utils/pre_checks/env.py", line 70, in check_env
    raise ValueError(
ValueError: Env must be of one of the following supported types: BaseEnv, gymnasium.Env, gym.Env, MultiAgentEnv, VectorEnv, RemoteBaseEnv, ExternalMultiAgentEnv, ExternalEnv, but instead is of type <class 'pettingzoo.utils.wrappers.order_enforcing.OrderEnforcingWrapper'>.

The above error has been found in your environment! We've added a module for checking your custom environments. It may cause your experiment to fail if your environment is not set up correctly. You can disable this behavior via calling `config.environment(disable_env_checking=True)`. You can run the environment checking module standalone by calling ray.rllib.utils.check_env([your env]).

# Train

In [ ]:
for i in range(stop_iters):
    print("== Iteration", i, "==")

    # improve the DQN policy
    print("-- DQN --")
    result_dqn = dqn.train()
    print(pretty_print(result_dqn))